In [42]:
import heapq
import random as rd
import numpy as np
import scipy.stats as sts

In [41]:
# code addapeted from session 5

class Event:
    '''
    Store the properties of one event in the Schedule class defined below. Each
    event has a time at which it needs to run, a function to call when running
    the event, along with the arguments and keyword arguments to pass to that
    function.
    '''
    def __init__(self, timestamp, function, *args, **kwargs):
        self.timestamp = timestamp
        self.function = function
        self.args = args
        self.kwargs = kwargs

    def __lt__(self, other):
        '''
        This overloads the less-than operator in Python. We need it so the
        priority queue knows how to compare two events. We want events with
        earlier (smaller) times to go first.
        '''
        return self.timestamp < other.timestamp

    def run(self, schedule):
        '''
        Run an event by calling the function with its arguments and keyword
        arguments. The first argument to any event function is always the
        schedule in which events are being tracked. The schedule object can be
        used to add new events to the priority queue.
        '''
        self.function(schedule, *self.args, **self.kwargs)


class Schedule:
    '''
    Implement an event schedule using a priority queue. You can add events and
    run the next event.
    
    The `now` attribute contains the time at which the last event was run.
    '''
    
    def __init__(self):
        self.now = 0  # Keep track of the current simulation time
        self.priority_queue = []  # The priority queue of events to run
    
    def add_event_at(self, timestamp, function, *args, **kwargs):
        # Add an event to the schedule at a particular point in time.
        heapq.heappush(
            self.priority_queue,
            Event(timestamp, function, *args, **kwargs))
    
    def add_event_after(self, interval, function, *args, **kwargs):
        # Add an event to the schedule after a specified time interval.
        self.add_event_at(self.now + interval, function, *args, **kwargs)
    
    def next_event_time(self):
        return self.priority_queue[0].timestamp

    def run_next_event(self):
        # Get the next event from the priority queue and run it.
        event = heapq.heappop(self.priority_queue)
        self.now = event.timestamp
        event.run(self)
        
    def __repr__(self):
        return (
            f'Schedule() at time {self.now} ' +
            f'with {len(self.priority_queue)} events in the queue')
    
    def print_events(self):
        print(repr(self))
        for event in sorted(self.priority_queue):
            print(f'   {event.timestamp}: {event.function.__name__}')

In [58]:
class Queue:
    def __init__(self):
        # We start with an empty queue and the server not busy
        self.people_in_queue = 0
        self.people_being_served = 0

    def add_customer(self):
        # Add the customer to the queue
        self.people_in_queue += 1
    
    def embark(self, n):
        self.people_in_queue -= n

class Bus:
    def __init__(self, initial_stop, 
                nStops=15, 
                time_moving_dist=sts.norm(loc=2,scale=0.5), 
                load_time_dist=sts.norm(loc = 0.03, scale = 0.01), 
                unload_time_dist=sts.norm(loc=0.05, scale = 0.01), 
                max_pass=130):

        self.nPassengers = 0
        self.cStop = initial_stop
        self.time_moving_dist = time_moving_dist
        self.max_pass = max_pass
        self.load_time_dist = load_time_dist
        self.unload_time_dist = unload_time_dist
        self.nStops = nStops
        self.stops = {i:0 for i in range(nStops)}

    def Load(self, busSystem, stop, schedule):
        n = busSystem.queues[stop]
        if n + self.nPassengers > self.max_pass:
            n = self.max_pass - self.nPassengers
        self.nPassengers += n
        busSystem.queues[stop] -= n
        for _ in range(n):
            trip = stop + round(rd.random()*7)
            if trip > self.nStops-1:
                trip -= self.nStops
            self.stops[stop] += 1
        busSystem.queues[stop].embark(n)
        schedule.add_event_after(
            self.time_moving_dist.rvs()+self.load_time_dist.rvs()*n,
            self.Unload, (busSystem, self.cStop+1))
    
    def Unload(self, busSystem, stop, schedule):
        if stop > self.nStops-1:
            stop -= self.nStops
        self.cStop = stop
        n = self.stops[stop]
        self.nPassengers -= n
        self.stops[stop] -= n
        schedule.add_event_after(
            self.unload_time_dist.rvs()*n,
            self.Load, busSystem, self.cStop)

class BusSystem:
    def __init__(self, n_busses, 
                arrival_distribution=sts.expon(scale=1),
                bus_interval_dist = sts.norm(loc=4, scale=0.5), 
                n_stops=15):

        self.queues = {i:Queue() for i in range(n_stops)}
        self.arrival_distribution = arrival_distribution
        self.n_stops = n_stops
        self.busses = {i:Bus(0) for i in range(n_busses)}
        self.bus_interval_dist = bus_interval_dist

    def add_customer(self, queue, schedule):
        # Add this customer to the queue
        self.queues[queue].add_customer()
        # Schedule when to add another customer
        schedule.add_event_after(
            self.arrival_distribution.rvs(),
            self.add_customer, (queue, schedule))

    def run(self, schedule):
        # Schedule when the first customer arrives
        for i in range(self.n_stops):
            schedule.add_event_after(
                self.arrival_distribution.rvs(),
                self.add_customer, (i, schedule))

        for i,bus in self.busses.items():
            schedule.add_event_after(
                self.bus_interval_dist.rvs()*(i+1),
                bus.Load, (self,0))
        

#change all this
def run_simulation(n_busses, run_until):
    schedule = Schedule()
    bus_system = BusSystem(n_busses)
    bus_system.run(schedule)
    while schedule.next_event_time() < run_until:
        schedule.run_next_event()
        print(bus_system.queues)
    return bus_system

In [59]:
run_simulation(1, 1000)

KeyError: Schedule() at time 0.03268440923665959 with 15 events in the queue